In [1]:
"""Hypothesis: Prices of housing went up post-covid"""


import geopy as gp
from geopy import distance
import pandas as pd

In [2]:
fn='/Users/mlid/Downloads/londonCoors.xlsx'

In [3]:
url_LondonHousePrices = "https://data.london.gov.uk/download/uk-house-price-index/70ac0766-8902-4eb5-aab5-01951aaed773/UK%20House%20price%20index.xls"

properties = pd.read_excel(url_LondonHousePrices, sheet_name='Average price', index_col= None)

In [4]:
df = pd.read_excel(fn)

In [5]:
print(df.head())

      Borough        Co-ordinates  Number
0  Hillingdon   51.5441, 0.4760°W      33
1      Harrow  51.5898°N 0.3346°W      32
2      Barnet  51.6252°N 0.1517°W      31
3     Enfield  51.6538°N 0.0799°W      30
4    Haringey  51.6000°N 0.1119°W      29


In [6]:
df=df.set_index('Number')

In [7]:
df=df.sort_index(ascending=True)

In [8]:
df

,Borough,Co-ordinates
Number,,
1,City of London,51.5155°N 0.0922°W
2,Westminster,51.4973°N 0.1372°W
3,Kensington & Chelsea,51.5020°N 0.1947°W
4,Hammersmith and Fulham,51.4927°N 0.2339°W
5,Wandsworth,51.4567°N 0.1910°W
6,Lambeth,51.4607°N 0.1163°W
7,Southwark,51.5035°N 0.0804°W
8,Tower Hamlets,51.5099°N 0.0059°W
9,Hackney,51.5450°N 0.0553°W


In [9]:
london=df.iloc[0,1]

In [10]:
df['points']=df['Co-ordinates'].apply(lambda p: gp.Point(p))

In [11]:
df['distFromLon (km)']=df['points'].apply(lambda p: gp.distance.geodesic(p,london).km)

In [12]:
df

,Borough,Co-ordinates,points,distFromLon (km)
Number,,,,
1,City of London,51.5155°N 0.0922°W,"51 30m 55.8s N, 0 5m 31.92s W",0.000000
2,Westminster,51.4973°N 0.1372°W,"51 29m 50.28s N, 0 8m 13.92s W",3.723168
3,Kensington & Chelsea,51.5020°N 0.1947°W,"51 30m 7.2s N, 0 11m 40.92s W",7.273068
4,Hammersmith and Fulham,51.4927°N 0.2339°W,"51 29m 33.72s N, 0 14m 2.04s W",10.160583
5,Wandsworth,51.4567°N 0.1910°W,"51 27m 24.12s N, 0 11m 27.6s W",9.481306
6,Lambeth,51.4607°N 0.1163°W,"51 27m 38.52s N, 0 6m 58.68s W",6.322540
7,Southwark,51.5035°N 0.0804°W,"51 30m 12.6s N, 0 4m 49.44s W",1.566402
8,Tower Hamlets,51.5099°N 0.0059°W,"51 30m 35.64s N, 0 0m 21.24s W",6.023360
9,Hackney,51.5450°N 0.0553°W,"51 32m 42s N, 0 3m 19.08s W",4.162848


In [13]:
dT=properties.T
dT=dT.rename(index={'Unnamed: 0': 'London_Borough'})
dTlocs=dT.index
dTcols=list(dT.columns.values)
dT.index=(dT[0])
dT[0]=dTlocs
dT.dropna()
relevantBs=dT[1:33]

In [18]:
relevantBs=relevantBs.set_index(0)

In [19]:
relevantBs

,1,2,3,4,5,6,7,8,9,10,...,336,337,338,339,340,341,342,343,344,345
0,,,,,,,,,,,,,,,,,,,,,
City of London,91448.98487,82202.77314,79120.70256,77101.20804,84409.14932,94900.51244,110128.0423,112329.4376,104473.1096,108038.1181,...,975240,963094,869039,930986,903718,958418,951649,931176,861107,807475
Barking & Dagenham,50460.2266,51085.77983,51268.96956,53133.50526,53042.24852,53700.34831,52113.12157,52232.19868,51471.61353,51513.7576,...,355596,353429,346193,345288,348254,349590,346099,343430,344171,345614
Barnet,93284.51832,93190.16963,92247.52435,90762.87492,90258.00033,90107.23471,91441.24768,92361.31512,93273.12245,92567.38498,...,592328,596085,602336,596064,587132,575141,575829,583479,597591,584740
Bexley,64958.09036,64787.92069,64367.49344,64277.66881,63997.13588,64252.32335,63722.70055,64432.60005,64509.54767,64529.93725,...,409059,402365,403398,399796,398544,391077,390880,396718,396278,396292
Brent,71306.56698,72022.26197,72015.76274,72965.63094,73704.04743,74310.48167,74127.03788,73547.0411,73789.54287,73264.05385,...,552767,551093,534129,525561,523992,524452,531478,531425,557552,565929
Bromley,81671.47692,81657.55944,81449.31143,81124.41227,81542.61561,82382.83435,82898.52264,82054.37156,81440.43008,81862.16087,...,512263,508434,500974,495896,497632,496393,497841,503888,509263,502753
Camden,120932.8881,119508.8622,120282.2131,120097.899,119929.2782,121887.4625,124027.5768,125529.8039,120596.8511,117458.4922,...,854762,858942,870793,855810,879927,859267,871822,838994,854042,810578
Croydon,69158.16225,68951.09542,68712.44341,68610.04641,68844.9169,69052.51103,69142.48112,68993.42545,69393.50023,68934.61234,...,429078,431797,428317,420441,412059,408859,406086,409938,416583,430541
Ealing,79885.89069,80897.06551,81379.86288,82188.90498,82077.05525,81630.66181,82352.2226,82706.65927,82011.08271,80522.66054,...,531259,531246,535352,522437,522810,522224,529437,535675,543218,554377


In [27]:
t2=df.merge(relevantBs,left_on='Borough',right_on=0,how='outer')

In [28]:
t2

,Borough,Co-ordinates,points,distFromLon (km),1,2,3,4,5,6,...,336,337,338,339,340,341,342,343,344,345
0,City of London,51.5155°N 0.0922°W,"51 30m 55.8s N, 0 5m 31.92s W",0.000000,91448.98487,82202.77314,79120.70256,77101.20804,84409.14932,94900.51244,...,975240,963094,869039,930986,903718,958418,951649,931176,861107,807475
1,Westminster,51.4973°N 0.1372°W,"51 29m 50.28s N, 0 8m 13.92s W",3.723168,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kensington & Chelsea,51.5020°N 0.1947°W,"51 30m 7.2s N, 0 11m 40.92s W",7.273068,182694.8326,182345.2463,182878.8231,184176.9168,191474.1141,197265.7602,...,1305382,1356424,1332696,1364904,1330033,1339650,1362492,1350811,1355950,1203055
3,Hammersmith and Fulham,51.4927°N 0.2339°W,"51 29m 33.72s N, 0 14m 2.04s W",10.160583,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wandsworth,51.4567°N 0.1910°W,"51 27m 24.12s N, 0 11m 27.6s W",9.481306,88559.04381,88641.01678,87124.81523,87026.00225,86518.05945,88114.3351,...,630568,630541,631835,625151,618574,618394,620574,623608,627230,637929
5,Lambeth,51.4607°N 0.1163°W,"51 27m 38.52s N, 0 6m 58.68s W",6.322540,67770.98843,65381.51908,66336.51868,66388.7716,69035.11076,68881.15764,...,550848,550491,551812,555124,556928,551514,540222,548939,561066,579990
6,Southwark,51.5035°N 0.0804°W,"51 30m 12.6s N, 0 4m 49.44s W",1.566402,67885.20344,64799.0648,65763.29719,63073.62117,64420.49933,64155.81449,...,554930,556336,541437,529503,518402,527497,536332,541457,541633,549329
7,Tower Hamlets,51.5099°N 0.0059°W,"51 30m 35.64s N, 0 0m 21.24s W",6.023360,59865.18995,62318.53353,63938.67686,66233.19383,66432.85846,66232.16372,...,488538,488206,490788,485043,485680,485554,479768,470178,474562,509454
8,Hackney,51.5450°N 0.0553°W,"51 32m 42s N, 0 3m 19.08s W",4.162848,61296.52637,63187.08332,63593.29935,65139.64403,66193.99212,66921.17101,...,622524,620395,614259,632267,630722,645012,644872,652928,634511,613851
9,Islington,51.5416°N 0.1022°W,"51 32m 29.76s N, 0 6m 7.92s W",2.985616,92516.48557,94342.37334,93465.86407,93344.49305,94346.39917,97428.94311,...,703076,697637,684105,686207,697986,714112,709118,712710,713652,718622


In [26]:
t1

,Borough,Co-ordinates,points,distFromLon (km),1,2,3,4,5,6,...,336,337,338,339,340,341,342,343,344,345
0,City of London,51.5155°N 0.0922°W,"51 30m 55.8s N, 0 5m 31.92s W",0.000000,91448.98487,82202.77314,79120.70256,77101.20804,84409.14932,94900.51244,...,975240,963094,869039,930986,903718,958418,951649,931176,861107,807475
1,Westminster,51.4973°N 0.1372°W,"51 29m 50.28s N, 0 8m 13.92s W",3.723168,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kensington & Chelsea,51.5020°N 0.1947°W,"51 30m 7.2s N, 0 11m 40.92s W",7.273068,182694.8326,182345.2463,182878.8231,184176.9168,191474.1141,197265.7602,...,1305382,1356424,1332696,1364904,1330033,1339650,1362492,1350811,1355950,1203055
3,Hammersmith and Fulham,51.4927°N 0.2339°W,"51 29m 33.72s N, 0 14m 2.04s W",10.160583,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wandsworth,51.4567°N 0.1910°W,"51 27m 24.12s N, 0 11m 27.6s W",9.481306,88559.04381,88641.01678,87124.81523,87026.00225,86518.05945,88114.3351,...,630568,630541,631835,625151,618574,618394,620574,623608,627230,637929
5,Lambeth,51.4607°N 0.1163°W,"51 27m 38.52s N, 0 6m 58.68s W",6.322540,67770.98843,65381.51908,66336.51868,66388.7716,69035.11076,68881.15764,...,550848,550491,551812,555124,556928,551514,540222,548939,561066,579990
6,Southwark,51.5035°N 0.0804°W,"51 30m 12.6s N, 0 4m 49.44s W",1.566402,67885.20344,64799.0648,65763.29719,63073.62117,64420.49933,64155.81449,...,554930,556336,541437,529503,518402,527497,536332,541457,541633,549329
7,Tower Hamlets,51.5099°N 0.0059°W,"51 30m 35.64s N, 0 0m 21.24s W",6.023360,59865.18995,62318.53353,63938.67686,66233.19383,66432.85846,66232.16372,...,488538,488206,490788,485043,485680,485554,479768,470178,474562,509454
8,Hackney,51.5450°N 0.0553°W,"51 32m 42s N, 0 3m 19.08s W",4.162848,61296.52637,63187.08332,63593.29935,65139.64403,66193.99212,66921.17101,...,622524,620395,614259,632267,630722,645012,644872,652928,634511,613851
9,Islington,51.5416°N 0.1022°W,"51 32m 29.76s N, 0 6m 7.92s W",2.985616,92516.48557,94342.37334,93465.86407,93344.49305,94346.39917,97428.94311,...,703076,697637,684105,686207,697986,714112,709118,712710,713652,718622
